### II. La Random Access Machine (RAM)

Une [RAM](https://en.wikipedia.org/wiki/Random-access_machine) est un modèle de calcul plus proche d'un ordinateur réel. Ce modèle de calcul étend les [machines à compteurs](https://fr.wikipedia.org/wiki/Machine_%C3%A0_compteurs) en ajoutant des registres la possibilité d'un addressage indirecte à registre. 

Notons que les machines à compteurs ont la même puissance de calcul que les machines de Turing (voir calculabilité). On peut donc simuler toute machine de Turing par une machine à deux compteurs, et inversement.

#### Ressources utiles

 * Machine Abstraite: https://fr.wikipedia.org/wiki/Machine_abstraite
 * Machine à compteurs: https://fr.wikipedia.org/wiki/Machine_%C3%A0_compteurs
 * Addressage indirect à registre: https://fr.wikipedia.org/wiki/Mode_d'adressage#Adressage_indirect_%C3%A0_registreindirect
 * Machine de Turing: https://fr.wikipedia.org/wiki/Machine_de_Turing
 * Random Access Machine: https://en.wikipedia.org/wiki/Random-access_machine
 * Architecture de Harvard:
 * Architecture de Von Neumann:
 * Cycle d'instructions: https://en.wikipedia.org/wiki/Instruction_cycle

In [5]:
%%bash

uv pip install --no-config numpy rich

Audited 2 packages in 2ms


In [2]:
"""This module provides an implementation of a Random Access Machine (RAM).

The RAM is a theoretical model of a computer that provides a simplified,
abstract view of a computer's operation. This implementation includes basic
arithmetic operations, data movement, and control flow operations.

Notes:
    The RAM Machine operates in cycles: fetch, decode, and execute. Each
    instruction goes through this cycle. The module uses the `numpy` library
    for numerical operations and the `rich` library for enhanced console
    printing.

Reference:
    - https://en.wikipedia.org/wiki/Random-access_machine
    - https://en.wikipedia.org/wiki/Instruction_set_architecture
"""
from __future__ import annotations

import time
from collections import defaultdict, deque
from dataclasses import dataclass, field
from enum import Enum
from pathlib import Path
from queue import Queue
from typing import Final, TextIO, Iterable, List, Tuple, Self

import numpy as np
from rich import print


COMMENT: Final[str] = ";"
INSTRUCTION_SEPARATOR: Final[str] = " "
BOOL32: Final[Tuple[np.uint32, np.uint32]] = (
    np.uint32(0x00000000),
    np.uint32(0xFFFFFFFF),
)
AT: Final[str] = "@"


def _conditional_branch(
    cond: bool, *, true: np.uint32, false: np.uint32
) -> np.uint32:
    """Determine the output value based on a given condition.

    The function evaluates the given condition and returns one of the two
    provided values `True` or `False` based on the result of the evaluation.

    Args:
        cond: The condition to evaluate.
        true: The value to return if the condition is True.
        false: The value to return if the condition is False.

    Returns:
        np.uint32: The value determined by the condition.
    """
    selector = BOOL32[cond]
    return (~selector & false) | (selector & true)


class Operation(str, Enum):
    """Enumeration of possible operations in the RAM.

    The enum provides a list of operations that the RAM machine can perform,
    ranging from basic arithmetic operations to data movement and control flow
    operations.

    Reference:
        - https://en.wikipedia.org/wiki/Instruction_set_architecture
    """

    READ = "READ"
    WRITE = "WRITE"
    SET = "SET"
    LOAD = "LOAD"
    STORE = "STORE"
    ADD = "ADD"
    SUB = "SUB"
    MUL = "MUL"
    DIV = "DIV"
    MOD = "MOD"
    INC = "INC"
    DEC = "DEC"
    JUMP = "JUMP"
    JUMZ = "JUMZ"
    JUML = "JUML"
    JUMG = "JUMG"
    HALT = "HALT"
    NOP = "NOP"


@dataclass
class Instruction:
    """Represents an instruction for the RAM.

    Each instruction is composed of an operation and an optional operand.
    The instruction can also be indirect, meaning the operand points to
    a memory location containing the actual operand value.

    Attributes:
        operation: The operation to be performed.
        operand: The operand for the operation. Can be a direct value or a
            memory address.
        indirect: Indicates if the instruction uses indirect addressing.

    Methods:
        decode: Class method to decode an instruction string
        into an Instruction instance.
    """

    operation: Operation = Operation.NOP
    operand: str = ""
    indirect: bool = False

    @classmethod
    def decode(cls, line: str) -> Self:
        """Decode a string into an Instruction instance.

        Args:
            line: The string to decode.

        Returns:
            The decoded instruction.
        """
        instruction_string, _sep, comment = line.strip().partition(COMMENT)
        if not instruction_string:
            # For convinience no instruction is a NOP.
            instruction_string = "NOP"

        operation, _sep, mod_operand = instruction_string.partition(
            INSTRUCTION_SEPARATOR
        )

        indirect = bool(mod_operand and (mod_operand[0] == AT))
        operand = mod_operand[int(indirect) :]

        return cls(
            operation=Operation(operation.rstrip().upper()),
            operand=operand.rstrip(),
            indirect=indirect,
        )


@dataclass
class RAM:
    """A Random Access Machine (RAM) implementation.

    The RAM is a theoretical model of a computer that provides a simplified,
    abstract view of a computer's operation. This implementation includes
    basic arithmetic operations, data movement, and control flow operations.

    Attributes:
        INPUT_BAND: The input band for feeding data into the machine.
        OUTPUT_BAND: The output band where the results are pushed.
        RAM: The main memory of the machine initialized with zeros.
        PROGRAM: The list of instructions to be executed by the machine.
        pc: Program Counter, which points to the next instruction to be
            executed.
        ACC: Accumulator register used to hold operands and results.
        mdr: Memory Data Register which holds the fetched instruction.
        cir: Current Instruction Register holding the decoded instruction.
        n: Operand of the current instruction.
        cnt: Control flag indicating if the machine should continue execution.

    Notes:
        The RAM Machine operates in cycles: fetch, decode, and execute.
        Each instruction goes through this cycle.

    Reference:
        - https://en.wikipedia.org/wiki/Random-access_machine
    """

    # IO
    INPUT_BAND: Final[Queue[int]] = field(default_factory=Queue)
    OUTPUT_BAND: Final[Queue[int]] = field(default_factory=Queue)

    # Memories
    RAM: Final[defaultdict[np.uint32, int]] = field(
        default_factory=lambda: defaultdict(int)
    )
    PROGRAM: Final[List[str]] = field(default_factory=lambda: [])

    # FLAGS register
    cnt: bool = True

    # Special registers
    pc: np.uint32 = np.uint32(0)  # Program Counter
    operand: np.uint32 = np.uint32(0)  # operand register
    mdr: str = ""  # memory data register
    cir: Instruction = field(  # Current instruction Register
        default_factory=lambda: Instruction()
    )

    # Register(s)
    @property
    def ACC(self) -> int:
        return self.RAM[0]

    @ACC.setter
    def ACC(self, value: int) -> Self:
        self.RAM[0] = value
        return self

    # Utilities
    def load_program(self, program: TextIO) -> None:
        """Load a program into the RAM.

        Args:
            program: The program file to be loaded.
        """
        self.PROGRAM.extend(program.readlines())

    def loads_program(self, program_path: Path) -> None:
        """Load a program into the RAM.

        Args:
            program: Path to the program file to be loaded.
        """
        with program_path.open("r", encoding="utf-8") as program:
            self.load_program(program)

    def set_inputs(self, inputs: Iterable[int]) -> None:
        """Set the input values for the RAM.

        Args:
            inputs: An iterable of input values.
        """
        for i in inputs:
            self.INPUT_BAND.put_nowait(i)

    def get_outputs(self) -> deque[int]:
        return list(self.OUTPUT_BAND.queue)

    # IO
    def read(self) -> None:
        """ACC ← INPUT[i++]
        
        We read an input, place it in ACC and shift the input ribbon.
        """
        self.ACC = (
            0 if self.INPUT_BAND.empty() else self.INPUT_BAND.get_nowait()
        )

    def write(self) -> None:
        """OUTPUT[i++] ← ACC

        We write ACC to output and shift the output ribbon.
        """
        self.OUTPUT_BAND.put_nowait(int(self.ACC))

    # RAM
    def set(self) -> None:
        """ACC ← operand"""
        self.ACC = self.operand

    def load(self) -> None:
        """ACC ← R[operand]"""
        self.ACC = self.RAM[self.operand]

    def store(self) -> None:
        """R[operand] ← ACC"""
        self.RAM[self.operand] = self.ACC

    # Arithmetic
    def add(self) -> None:
        """ACC ← ACC + R[operand]"""
        self.ACC += self.RAM[self.operand]

    def sub(self) -> None:
        """ACC ← ACC - R[operand]"""
        self.ACC -= self.RAM[self.operand]

    def mul(self) -> None:
        """ACC ← ACC * R[operand]"""
        self.ACC *= self.RAM[self.operand]

    def div(self) -> None:
        """ACC ← ACC // R[operand]"""
        self.ACC //= self.RAM[self.operand]

    def mod(self) -> None:
        """ACC ← ACC % R[operand]"""
        self.ACC %= self.RAM[self.operand]

    def inc(self) -> None:
        """R[operand] ← R[operand] + 1"""
        self.RAM[self.operand] += 1

    def dec(self) -> None:
        """R[operand] ← R[operand] - 1"""
        self.RAM[self.operand] -= 1

    # Control Flow
    def jump(self) -> None:
        """Jump to the line 'operand'"""
        self.pc = self.operand

    def jumz(self) -> None:
        """Jump to the line 'operand' if the condition register is 0"""
        self.pc = _conditional_branch(
            self.ACC == 0,
            true=self.operand,
            false=self.pc,
        )

    def juml(self) -> None:
        """Jump to the line 'operand' if the condition register is < 0"""
        self.pc = _conditional_branch(
            self.ACC < 0,
            true=self.operand,
            false=self.pc,
        )

    def jumg(self) -> None:
        """Jump to the line 'operand' if the condition register is > 0"""
        self.pc = _conditional_branch(
            self.ACC > 0,
            true=self.operand,
            false=self.pc,
        )

    def halt(self) -> None:
        """Stop the RAM"""
        self.cnt = False

    def nop(self) -> None:
        """Do nothing"""
        pass

    # Fetch-Decode-Execute: https://en.wikipedia.org/wiki/Instruction_cycle
    def fetch(self) -> None:
        self.mdr = self.PROGRAM[self.pc]
        self.pc += 1

    def decode(self) -> None:
        self.cir = Instruction.decode(self.mdr)
        if self.cir.operand:
            self.operand = _conditional_branch(
                self.cir.indirect,
                true=np.uint32(np.int32(
                    self.RAM[np.uint32(np.int32(self.cir.operand))]
                )),
                false=np.uint32(np.int32(self.cir.operand)),
            )

    def execute(self) -> None:
        getattr(self, self.cir.operation.lower())()

    def instruction_cycle(self) -> None:
        self.fetch()
        self.decode()
        self.execute()

    def run(self, freq: float) -> int:
        """Run the RAM until it reaches a HALT instruction.
        
        Args:
            freq: the frequency (speed) of computation of the RAM.

        Returns:
            The number of cycles perform until the machine halts.
        """
        cycles = 0
        max_nap_time = 1 / freq
        while self.cnt:
            work_time = -time.monotonic()
            self.instruction_cycle()
            cycles += 1
            work_time += time.monotonic()
            time.sleep(max(max_nap_time - work_time, 0))

        return cycles

### Programation de la RAM

#### 📖 Exercice

 * Executer les  RAM suivantes

In [5]:
from io import StringIO  # define an IO (file like) from a string.

ram = RAM()
ram.load_program(StringIO("""; Do nothing
HALT; final instruction
"""))
ram.set_inputs([8, 7])
ram.run(freq=100)

print(ram.get_outputs())

[]

In [6]:
from io import StringIO  # define an IO (file like) from a string.

ram = RAM()
ram.load_program(StringIO("""; Write 32 to the output
SET 32      ; 1. ACC ← 32
WRITE       ; 2. OUTPUT[j++] ← ACC
HALT        ; 3. STOP
"""))
ram.set_inputs([])
ram.run(freq=100)

print(ram.get_outputs())

[32]

In [7]:
from io import StringIO  # define an IO (file like) from a string.

ram = RAM()
ram.load_program(StringIO("""; Add 32 to an input
SET 32      ; 1. ACC ← 32
STORE 1     ; 2. R[1] ← ACC
READ        ; 3. ACC ← INPUT[i++]
ADD 1       ; 4. ACC ← ACC + R[1]
WRITE       ; 5. OUTPUT[j++] ← ACC
HALT        ; 6. STOP
"""))
ram.set_inputs([3])
ram.run(freq=100)

print(ram.get_outputs())

[35]

#### 📖 Exercice

 * Écrire un programme qui lit un nombre sur le ruban d'entrée et le reporte sur le ruban de sortie

In [11]:
from io import StringIO  # define an IO (file like) from a string.

ram = RAM()
ram.load_program(StringIO("""; Copy an input ribbon value to the output ribbon.
READ         ; 1. ACC ← INPUT[i++]
WRITE        ; 2. OUTPUT[j++] ← ACC
HALT         ; 3. STOP
"""))
ram.set_inputs([8, 7])
ram.run(freq=100)

print(ram.get_outputs())

[8]

#### 📖 Exercice

 * Écrire un programme qui calcule le maximum de deux nombres

In [16]:
from io import StringIO  # define an IO (file like) from a string.

ram = RAM()
ram.load_program(StringIO("""; Compute the maximum of two integer.
        READ        ;  1. ACC ← INPUT[i++]
        STORE 1     ;  2. R[1] ← ACC
        READ        ;  3. ACC ← INPUT[i++]
        STORE 2     ;  4. R[2] ← ACC
        SUB 1       ;  5. ACC ← ACC - R[1]
        JUMG 10     ;  6. IF (R[2] - R[1] > 0) JUMP TO INSTRUCTION #10
        LOAD 1      ;  7. ACC ← R[1]
        WRITE       ;  8. OUTPUT[j++] ← ACC
        HALT        ;  9. STOP
        LOAD 2      ; 10. ACC ← R[2]
        WRITE       ; 11. OUTPUT[j++] ← ACC
        HALT        ; 12. STOP
"""))
ram.set_inputs([4, 92])
ram.run(freq=100)

print(ram.get_outputs())

[92]

#### 📖 Exercice

 * [Difficile] Écrire un programme qui calcule le maximum d'une liste de nombres entiers strictements positifs.

In [15]:
from io import StringIO  # define an IO (file like) from a string.

ram = RAM()
ram.load_program(
    StringIO("""; Compute the maximum of a list of strictly positive integer.
        READ        ;  1. ACC ← INPUT[i++]
        STORE 1     ;  2. R[1] ← ACC
        READ        ;  3. ACC ← INPUT[i++]
        JUMZ 12     ;  4. IF ACC = 0 JUMP TO INSTRUCTION #12
        STORE 2     ;  5. R[2] ← ACC
        LOAD 1      ;  6. ACC ← R[1]
        SUB 2       ;  7. ACC ← ACC - R[2]
        JUMG 3      ;  8. IF (R[1] - R[2] > 0) JUMP TO INSTRUCTION #3
        LOAD 2      ;  9. ACC ← R[2]
        STORE 1     ; 10. R[1] ← ACC
        JUMP 3      ; 11. JUMP TO INSTRUCTION #3
        LOAD 1      ; 12. ACC ← R[1]
        WRITE       ; 13. OUTPUT[j++] ← ACC
        HALT        ; 14. STOP
        """)
)
ram.set_inputs([1, 2, 3, 2, 7, 8, 2, 1])
ram.run(freq=100)

print(ram.get_outputs())

[8]